# 函数装饰器和闭包

## 7.5 闭包 

In [12]:
# callable object
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [13]:
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [14]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [15]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [16]:
avg.__code__.co_varnames

('new_value', 'total')

In [17]:
avg.__code__.co_freevars

('series',)

In [18]:
avg.__closure__

(<cell at 0x000002184F68D060: list object at 0x000002184FFB5000>,)

In [19]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## 7.6 nonlocal声明

In [20]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1 
        total += new_value
        return total / count

    return averager


In [21]:
avg = make_averager()
print(avg(10))

UnboundLocalError: local variable 'count' referenced before assignment

In [22]:
# a right way
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
        
    return averager

In [23]:
avg = make_averager()
avg(10)

10.0

## 7.7 实现一个简单的装饰器

In [24]:
import time 

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result}')
        return result
    return clocked

In [25]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)


In [26]:
print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.13112440s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000060s] factorial(1) -> 1
[0.00001400s] factorial(2) -> 2
[0.00002200s] factorial(3) -> 6
[0.00002860s] factorial(4) -> 24
[0.00003590s] factorial(5) -> 120
[0.00004430s] factorial(6) -> 720
6! =  720


In [27]:
factorial.__name__

'clocked'

In [28]:
import time 
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0 
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = [f'{k}={w}' for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result}')
        return result
    return clocked

## 7.8 标准库中的装饰器

In [29]:
@clock
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

In [30]:
print(fibonacci(6))

[0.00000040s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00006300s] fibonacci(2) -> 1
[0.00000030s] fibonacci(1) -> 1
[0.00000040s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00001030s] fibonacci(2) -> 1
[0.00002110s] fibonacci(3) -> 2
[0.00009740s] fibonacci(4) -> 3
[0.00000010s] fibonacci(1) -> 1
[0.00000030s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00000890s] fibonacci(2) -> 1
[0.00001790s] fibonacci(3) -> 2
[0.00000020s] fibonacci(0) -> 0
[0.00000030s] fibonacci(1) -> 1
[0.00002030s] fibonacci(2) -> 1
[0.00000020s] fibonacci(1) -> 1
[0.00000030s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00000950s] fibonacci(2) -> 1
[0.00001910s] fibonacci(3) -> 2
[0.00004900s] fibonacci(4) -> 3
[0.00007630s] fibonacci(5) -> 5
[0.00018350s] fibonacci(6) -> 8
8


In [31]:
import functools

# functools.lru_cache(maxsize=128, typed=False)
@functools.lru_cache()
@clock
def fibonacci(n):
    return n if n < 2 else fibonacci(n-2) + fibonacci(n-1)

In [32]:
print(fibonacci(6))

[0.00000040s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00007950s] fibonacci(2) -> 1
[0.00000060s] fibonacci(3) -> 2
[0.00009120s] fibonacci(4) -> 3
[0.00000050s] fibonacci(5) -> 5
[0.00010330s] fibonacci(6) -> 8
8


## 7.9 叠放装饰器

## 7.10 参数化装饰器

In [37]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s) -> decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate


In [38]:
@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False) -> decorate(<function f1 at 0x000002184FE67EB0>)
running register(active=True) -> decorate(<function f2 at 0x000002184FE67C70>)


In [39]:
registry

{<function __main__.f2()>}

In [41]:
register()(f3)

running register(active=True) -> decorate(<function f3 at 0x000002184FE66D40>)


<function __main__.f3()>

In [42]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [43]:
register(active=False)(f2)

running register(active=False) -> decorate(<function f2 at 0x000002184FE67C70>)


<function __main__.f2()>

In [44]:
registry

{<function __main__.f3()>}

In [45]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorater(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0 
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorater

In [46]:
@clock()
def snooze(seconds):
    time.sleep(seconds)

In [48]:
for i in range(3):
    snooze(.123)

[0.12915730s] snooze(0.123) -> None
[0.12508870s] snooze(0.123) -> None
[0.12538050s] snooze(0.123) -> None


In [49]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

In [50]:
for i in range(3):
    snooze(.123)

snooze: 0.1345858999993652s
snooze: 0.1269473999273032s
snooze: 0.13720460003241897s


In [51]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

In [52]:
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.129s
snooze(0.123) dt=0.124s
snooze(0.123) dt=0.138s
